In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls
%cd gdrive/My Drive/final-project/collab

gdrive	sample_data
/content/gdrive/My Drive/final-project/collab


In [3]:
!pip install transformers

     |████████████████████████████████| 1.3MB 11.3MB/s 
     |████████████████████████████████| 2.9MB 48.9MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 
     |████████████████████████████████| 890kB 45.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=88694367b14ed98ce3a8484849691a6eaae09e1dd068784d331b0ac20fea0d9e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
# specify GPU
device = torch.device("cuda")
torch.cuda.get_device_name(0)

'Tesla T4'

In [5]:
import preprocess as pproc
import dataprocess as dproc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
df = pproc.load_data("./dataset.csv")

Number of training rows: 94,471

legit    0.554879
spam     0.445121
Name: tag, dtype: float64

Drop 680 row with null value

Number of remain rows: 93,791

Clean data        ... Elapsed time: 149.502


In [7]:
MAX_LEN = 512
# define a batch size
batch_size = 32
# learning rate
learning_rate = 2e-5
epochs = 4

In [8]:
model, optimizer, class_wts, train_dataloader, val_dataloader, test_dataloader = dproc.data_processing(df, MAX_LEN, learning_rate, batch_size)

Load model        ...



Elapsed time: 9.603
Split data        ... Elapsed time: 0.063
Tokenize          ... Elapsed time: 53.296
Seq to Tensor     ... Elapsed time: 4.808
Create DataLoader ... Elapsed time: 0.000


In [9]:
import torch.nn as nn

# push the model to GPU
model = model.to(device)

# convert class weight to tensor
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights)

In [ ]:
from timeit import default_timer as timer

best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
    
  print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
  start = timer()
  #train model
  train_loss, _ = dproc.train(device, model, optimizer, cross_entropy, train_dataloader)
  
  #evaluate model
  valid_loss, _ = dproc.evaluate(device, model, cross_entropy, val_dataloader)
  
  #save the best model
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), './saved_weights.pt')
  
  # append training and validation loss
  train_losses.append(train_loss)
  valid_losses.append(valid_loss)
  
  print(f'\nTraining Loss: {train_loss:.3f}')
  print(f'Validation Loss: {valid_loss:.3f}')
  print("Epoch time: {:.5f}".format(timer()-start))